In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = (SparkSession
        .Builder()
        .appName("Python Spark SQL basic example")
        .master("local[2]")
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
)
spark.sparkContext.setLogLevel('WARN')

:: loading settings :: url = jar:file:/usr/local/sdkman/candidates/spark/3.3.2/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6e1b7d74-a239-4e37-86aa-e7f2e680381e;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;2.7.0 in central
	found org.apache.hadoop#hadoop-common;2.7.0 in central
	found org.apache.hadoop#hadoop-annotations;2.7.0 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found xmlenc#xmlenc;0.52 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.4 in central
	found commons-io#commons-io;2.4 in central
	found commons-net#commons-net;3.1 in central
	found commons-collections#commons-collect

23/05/29 20:54:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
#spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set("spark.sql.execution.pythonUDF.arrow.enabled", "false")

# 1. Song Data

In [6]:
df_songs = spark.read.json("./data/song_data/A/*/*/*.json")
df_songs.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



The infered schema looks good. We can create one to make it more explicit

In [7]:
from pyspark.sql.types import (
    StructType,
    StructField, 
    StringType, 
    DoubleType,
    LongType,
    IntegerType,
    FloatType
)
songs_schema = StructType([
    StructField('artist_id', StringType()),
    StructField('artist_latitude', FloatType()),
    StructField('artist_location', StringType()),
    StructField('artist_longitude', FloatType()),
    StructField('artist_name', StringType()),
    StructField('duration', FloatType()),
    StructField('num_songs', IntegerType()),
    StructField('song_id', StringType()),
    StructField('title', StringType()),
    StructField('year', IntegerType())
])

In [8]:
df_songs = spark.read.json("./data/song_data/A/*/*/*.json", 
                           schema = songs_schema)

## 1.1 Songs Table

In [9]:
df_songs.limit(5).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR4T2IF1187B9ADBB7,63.96027,"<a href=""http://billyidol.net"" onmousedown='Un...",10.22442,Billy Idol,233.220764,1,SOVIYJY12AF72A4B00,The Dead Next Door (Digitally Remastered 99),1983
1,AR4T2IF1187B9ADBB7,63.96027,"<a href=""http://billyidol.net"" onmousedown='Un...",10.22442,Billy Idol,287.921173,1,SOVYXYL12AF72A3373,Rebel Yell (1999 Digital Remaster),1983
2,ARQ846I1187B9A7083,NaN,,NaN,Yvonne S. Moriarty / Walt Fowler / Ladd McInto...,196.048523,1,SOEPTVC12A67ADD0DA,"To Zucchabar [""Gladiator"" - Music from the Mot...",0
3,AR4T2IF1187B9ADBB7,63.96027,"<a href=""http://billyidol.net"" onmousedown='Un...",10.22442,Billy Idol,247.535873,1,SOLQYSZ12AB0181F97,Mony Mony (Live),1987
4,AR3TZ691187FB3DBB1,NaN,,NaN,Russell Watson / Pino Palladino / Robbie McInt...,273.449341,1,SOVPFJK12A6701CB16,Barcelona - (Friends until the end),2000


In [12]:
songs_table = (
    df_songs.
    select('song_id', 'title', 'artist_id', 'year', 'duration')
    .drop_duplicates(["song_id"])
)

songs_table.limit(5).toPandas()

In [ ]:
songs_table.write.parquet(path = "./songs_table.parquet",
                          partitionBy= ["year", "artist_id"])

## 2. Artist Table


In [14]:
artists_table = (
    df_songs
    .select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude")
    .withColumnRenamed("artist_name", "name")
    .drop_duplicates(["artist_id"])
)
artists_table.limit(5).toPandas()

,artist_id,name,artist_location,artist_latitude,artist_longitude
0,AR00B1I1187FB433EB,Eagle-Eye Cherry,"Stockholm, Sweden",NaN,NaN
1,AR00FVC1187FB5BE3E,Panda,"Monterrey, NL, México",25.670839,-100.309532
2,AR00LNI1187FB444A5,Bruce BecVar,,NaN,NaN
3,AR00TGQ1187B994F29,Paula Toller,,NaN,NaN
4,AR016P51187B98E398,Indian Ropeman,,NaN,NaN


In [16]:
artists_table.write.parquet(path = "./artists_table.parquet")

# 2. Log Data

In [17]:
df_logs = spark.read.json("./data/log_data/2018/11/*.json")
df_logs.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [18]:
df_logs.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12


Some datatypes are not accurate, so, we can
* Fix them individually
* Or enforce a schema to fix them when reading the data

In [19]:
from pyspark.sql.types import (
    StructType,
    StructField, 
    StringType, 
    DoubleType,
    LongType,
    TimestampType,
    IntegerType
)
# # We want a schema like this one
#  |-- artist: string (nullable = true)
#  |-- auth: string (nullable = true)
#  |-- firstName: string (nullable = true)
#  |-- gender: string (nullable = true)
#  |-- itemInSession: long (nullable = true)
#  |-- lastName: string (nullable = true)
#  |-- length: double (nullable = true)
#  |-- level: string (nullable = true)
#  |-- location: string (nullable = true)
#  |-- method: string (nullable = true)
#  |-- page: string (nullable = true)
#  |-- registration: double (nullable = true)
#  |-- sessionId: long (nullable = true)
#  |-- song: string (nullable = true)
#  |-- status: long (nullable = true)
#  |-- ts: long (nullable = true)
#  |-- userAgent: string (nullable = true)
#  |-- userId: string (nullable = true)
logs_schema = StructType([
    StructField("artist", StringType()),
    StructField("auth", StringType()),
    StructField("firstName", StringType()),
    StructField("gender", StringType()),
    StructField("itemInSession", LongType()),
    StructField("lastName", StringType()),
    StructField("length", DoubleType()),
    StructField("level", StringType()),
    StructField("location", StringType()),
    StructField("method", StringType()),
    StructField("page", StringType()),
    StructField("registration", TimestampType()),
    StructField("sessionId", LongType()),
    StructField("song", StringType()),
    StructField("status", LongType()),
    StructField("ts", TimestampType()),
    StructField("userAgent", StringType()),
    StructField("userId", IntegerType())
])

In [20]:
df_logs = spark.read.json("./data/log_data/2018/11/*.json",
                         schema=logs_schema)


In [21]:
df_logs.limit(5).toPandas()

ValueError: year 50841 is out of range

It looks like we need to get our hands a little bit dirty. A quick fix using the schema didn't do it

In [25]:
df_logs = spark.read.json("./data/log_data/2018/11/*.json")

In [26]:
from pyspark.sql.types import IntegerType
df_logs = (
    df_logs
    .withColumn('ts',  (df_logs["ts"].cast('float')/1000).cast('timestamp'))
    .withColumn('registration', (df_logs["registration"].cast('float')/1000).cast('timestamp'))
    .withColumn('userId', df_logs["userId"].cast(IntegerType()))
)

df_logs.limit(3).toPandas()

/workspaces/spark-project/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/workspaces/spark-project/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,2018-10-31 20:10:49.728,583,Sehr kosmisch,200,2018-11-15 00:29:39.712,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,2018-10-31 20:10:49.728,583,The Big Gundown,200,2018-11-15 00:40:35.072,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,2018-10-31 20:10:49.728,583,Marry Me,200,2018-11-15 00:44:57.216,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [27]:
# renaming from camel case to  snake case
df_logs = (
    df_logs
    .withColumnRenamed('firstName', 'first_name')
    .withColumnRenamed('lastName', 'last_name')
    .withColumnRenamed('userId', 'user_id')
    .withColumnRenamed('itemInSession','item_in_session')
    .withColumnRenamed('sessionId','session_id')
    .withColumnRenamed('userAgent', 'user_agent')
)
df_logs.limit(3).toPandas()

/workspaces/spark-project/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/workspaces/spark-project/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,artist,auth,first_name,gender,item_in_session,last_name,length,level,location,method,page,registration,session_id,song,status,ts,user_agent,user_id
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,2018-10-31 20:10:49.728,583,Sehr kosmisch,200,2018-11-15 00:29:39.712,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,2018-10-31 20:10:49.728,583,The Big Gundown,200,2018-11-15 00:40:35.072,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,2018-10-31 20:10:49.728,583,Marry Me,200,2018-11-15 00:44:57.216,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


## 2.1 Songplay table

* songplays - records in log data associated with song plays i.e. records with page `NextSong`
    * songplay_id, 
    * start_time, 
    * user_id, 
    * level, 
    * song_id, 
    * artist_id, 
    * session_id, 
    * location, 
    * user_agent

In [28]:
import pyspark.sql.functions as f

In [29]:
songplay_table = (
    df_logs
    .filter(f.lower(df_logs["page"]) == "nextsong")
    .select('ts', 'user_id',  'level', 'song', 'artist', 'session_id', 'location', 'user_agent')
    .join(df_songs, on = df_logs["song"] == df_songs["title"], how = "inner")
    .withColumnRenamed("ts", "start_time")
    .withColumn('songplay_id', f.monotonically_increasing_id())
)
songplay_table.limit(5).toPandas()

/workspaces/spark-project/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,start_time,user_id,level,song,artist,session_id,location,user_agent,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year,songplay_id
0,2018-11-21 08:25:43.168,88,paid,Die Kunst der Fuge_ BWV 1080 (2007 Digital Rem...,Lionel Rogg,744,"Sacramento--Roseville--Arden-Arcade, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",ARN8NCB1187FB49652,46.208351,"Geneva, Switzerland",6.142700,Lionel Rogg,172.381592,1,SOCHPTV12A6BD53113,Die Kunst der Fuge_ BWV 1080 (2007 Digital Rem...,0,8589934592
1,2018-11-29 16:58:07.744,49,paid,Fighters (feat. Matthew Santos) (Amended Album...,Lupe Fiasco feat. Matthew Santos,1041,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,ARPFHN61187FB575F6,41.884151,"Chicago, IL",-87.632408,Lupe Fiasco feat. Matthew Santos,213.341583,1,SOGXSWA12A6D4FBC99,Fighters (feat. Matthew Santos) (Amended Album...,0,25769803776
2,2018-11-28 08:18:36.160,58,paid,"Suena (""Some Day"" end title song ""The Hunchbac...",Luis Miguel,887,"Augusta-Richmond County, GA-SC","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",ARM6T8I1187FB36CC8,NaN,"San Juan, Puerto Rico",NaN,Luis Miguel,256.130157,1,SOJWCWM12A8C13B664,"Suena (""Some Day"" end title song ""The Hunchbac...",1996,34359738368
3,2018-11-28 23:33:55.328,24,paid,It's Not Easy (Being Green) (Featuring Pierre ...,MC Lars,984,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",ARHQBRZ1187FB3BDA2,NaN,"Berkeley, California",NaN,MC Lars,222.876282,1,SOHRHCN12AB018B0F4,It's Not Easy (Being Green) (Featuring Pierre ...,2009,34359738369
4,2018-11-26 15:36:17.664,88,paid,Shimmy Shimmy Quarter Turn (Take It Back To Sq...,Hellogoodbye,900,"Sacramento--Roseville--Arden-Arcade, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",ARCE0IX1187FB528B4,NaN,"Huntington Beach, CA",NaN,Hellogoodbye,193.932617,1,SOARTQC12A58A77F0C,Shimmy Shimmy Quarter Turn (Take It Back To Sq...,2004,68719476736


## 2.2 Users Table

users - users in the app
* user_id,
* first_name, 
* last_name, 
* gender, 
* level

### We MUST use the user's entry with the latest timestamp
* That's why we're grouping by `user_id`
* Then, aggregating by maximum (latest) `ts`

In [30]:
users_events = (
    df_logs
    .filter(df_logs['user_id'].isNotNull())
    .select('user_id', 'first_name', 'last_name', 'gender', 'level', 'ts')
)

max_timestamps = (
    users_events
    .groupBy('user_id')
    .agg(f.max('ts').alias('max_ts'))
)

users_table = (
    users_events
    .join(max_timestamps, on = "user_id", how = "inner")
    .select('user_id', 'first_name', 'last_name', 'gender', 'level')
)

users_table.limit(5).toPandas()

,user_id,first_name,last_name,gender,level
0,26,Ryan,Smith,M,free
1,26,Ryan,Smith,M,free
2,26,Ryan,Smith,M,free
3,9,Wyatt,Scott,M,free
4,12,Austin,Rosales,M,free


## 2.3 Time Table
time - timestamps of records in songplays broken down into specific units
* start_time, 
* hour, 
* day, 
* week, 
* month, 
* year, 
* weekday

In [32]:
# we want the time to include
# hour, day, week, month, year, weekday
time_table = (
    df_logs
    .select('ts')
    .filter(df_logs['ts'].isNotNull())
    .withColumnRenamed('ts', 'start_time')
    .withColumn('hour', f.hour('start_time'))
    .withColumn('day', f.dayofmonth('start_time'))
    .withColumn('week', f.weekofyear('start_time'))
    .withColumn('month', f.month('start_time'))
    .withColumn('year', f.year('start_time'))
    .withColumn('weekday', f.dayofweek('start_time'))
)

time_table.limit(5).toPandas()

/workspaces/spark-project/.venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,start_time,hour,day,week,month,year,weekday
0,2018-11-15 00:29:39.712,0,15,46,11,2018,5
1,2018-11-15 00:40:35.072,0,15,46,11,2018,5
2,2018-11-15 00:44:57.216,0,15,46,11,2018,5
3,2018-11-15 01:57:02.592,1,15,46,11,2018,5
4,2018-11-15 03:28:47.616,3,15,46,11,2018,5
